## 한글 폰트 설치

- 참고: https://teddylee777.github.io/colab/colab-korean

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## GoogleMaps 패키지 설치

- Google Cloud API 사용 절차 등록
- https://teddylee777.github.io/data_science/geocoding

In [ ]:
# googlemaps 설치
# !pip install googlemaps

In [1]:
private_key = # Google Cloud API 키를 입력합니다.
private_key = open('GOOGLE_MAP_API_KEY.txt', 'r')
google_map_key = private_key.readline()

# Google Cloud API 키
google_map_key[:10]

'AIzaSyBfu3'

`googlemaps` 패키지 import 

In [2]:
import googlemaps
import os

# maps 객체 생성
maps = googlemaps.Client(key=google_map_key)
maps

**지역 명칭**을 직접 입력하여 위/경도 좌표계를 구할 수 있습니다.

- 명칭 입력시 return은 `json` 리스트를 반환합니다.

In [3]:
# 코드입력
results = maps.geocode('판교역')
results

[{'address_components': [{'long_name': 'Pangyo Station',
    'short_name': 'Pangyo Station',
    'types': ['establishment',
     'point_of_interest',
     'subway_station',
     'transit_station']},
   {'long_name': '160', 'short_name': '160', 'types': ['premise']},
   {'long_name': 'Pangyoyeok-ro',
    'short_name': 'Pangyoyeok-ro',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': 'Sampyeong-dong',
    'short_name': 'Sampyeong-dong',
    'types': ['political', 'sublocality', 'sublocality_level_2']},
   {'long_name': 'Bundang-gu',
    'short_name': 'Bundang-gu',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Seongnam-si',
    'short_name': 'Seongnam-si',
    'types': ['locality', 'political']},
   {'long_name': 'Gyeonggi-do',
    'short_name': 'Gyeonggi-do',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'South Korea',
    'short_name': 'KR',
    'types': ['country', 'political']},
  

`results`로부터 위/경도 데이터를 추출합니다.

In [4]:
for result in results:
    print(result['geometry']['location'])
    break
    
lat, lng = result['geometry']['location']['lat'], result['geometry']['location']['lng']

{'lat': 37.394776, 'lng': 127.11116}


## folium을 사용한 지도 시각화

In [5]:
import folium


m = folium.Map(location=[lat, lng],
               zoom_start=17, 
               width=750, 
               height=500
              )

folium.Marker([lat, lng],
              popup="판교역",
              tooltip="판교역 입구").add_to(m)
m

위의 기능들을 하나로 묶어 함수를 생성합니다.

In [6]:
def viz_map(place):
    # 코드입력
    results = maps.geocode(place)
    
    lat, lng = None, None
    
    for result in results:
        lat, lng = result['geometry']['location']['lat'], result['geometry']['location']['lng']
        print(f'위도: {lat:.3f}, 경도: {lng:.3f}')
        break
    
    if lat is None:
        return '검색 결과를 찾을 수 없습니다.'

    # folium 시각화
    m = folium.Map(location=[lat, lng],
                   zoom_start=17, 
                   width=750, 
                   height=500
                  )

    folium.Marker([lat, lng],
                  popup=place).add_to(m)
    return m

In [7]:
viz_map('종각역')

위도: 37.570, 경도: 126.983


## 공공데이터 다운로드 및 주소 데이터 전처리

In [8]:
import pandas as pd
import numpy as np
import os

DATA = 'data'

[서울교통공사 역주소 데이터 다운로드](http://data.seoul.go.kr/dataList/OA-12035/S/1/datasetView.do;jsessionid=7C7AC42AD377ADC8D042A39DAF162243.new_portal-svr-11)

In [36]:
from teddynote import dataset

dataset.download('역주소전화번호')

======= 다운로드 시작 =======

data\서울교통공사_역주소.xlsx


  0%|          | 0.00/29.6k [00:00<?, ?B/s]


======= 다운로드 완료 =======


In [37]:
filepath = os.path.join(DATA, '서울교통공사_역주소.xlsx')
filepath

'data\\서울교통공사_역주소.xlsx'

`서울교통공사_역주소.xlsx` 데이터셋을 로드합니다.

In [38]:
address = pd.read_excel(filepath)
address.head()

연번  역번호  호선    역명         역전화번호                           역주소
0   1  150   1   서울역  02-6110-1331  서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1   2  151   1    시청  02-6110-1321     서울특별시 중구 세종대로 지하 101 (정동)
2   3  152   1    종각  02-6110-1311     서울특별시 종로구 종로 지하 55 (종로1가)
3   4  153   1  종로3가  02-6110-1301    서울특별시 종로구 종로 지하 129 (종로3가)
4   5  154   1  종로5가  02-6110-1291   서울특별시 종로구 종로 지하 216  (종로5가)

**역명** 중복 값을 제거합니다.

In [39]:
# 코드입력
address = address.drop_duplicates('역명')
address

연번   역번호  호선      역명         역전화번호                           역주소
0      1   150   1     서울역  02-6110-1331  서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1      2   151   1      시청  02-6110-1321     서울특별시 중구 세종대로 지하 101 (정동)
2      3   152   1      종각  02-6110-1311     서울특별시 종로구 종로 지하 55 (종로1가)
3      4   153   1    종로3가  02-6110-1301    서울특별시 종로구 종로 지하 129 (종로3가)
4      5   154   1    종로5가  02-6110-1291   서울특별시 종로구 종로 지하 216  (종로5가)
..   ...   ...  ..     ...           ...                           ...
285  286  4132   9    석촌고분  02-2656-0932             서울특별시 송파구 삼학사로 53
287  288  4134   9    송파나루  02-2656-0934             서울특별시 송파구 오금로 165
288  289  4135   9    한성백제  02-2656-0935            서울특별시 송파구 위례성대로 51
290  291  4137   9    둔촌오륜  02-2656-0937            서울특별시 송파구 강동대로 327
291  292  4138   9  중앙보훈병원  02-2656-0938             서울특별시 강동구 명일로 117

[253 rows x 6 columns]

**역주소** 데이터를 `geocode`에 적용하여 위/경도 좌표계를 구합니다.

In [40]:
address['역주소'][0]

'서울특별시 중구 세종대로 지하 2 (남대문로 5가)'

In [19]:
viz_map(address['역주소'][0])

위도: 37.558, 경도: 126.975


역주소 끝에는 괄호안에 동이 같이 표기되어 있습니다.

In [20]:
station_list = list(address['역주소'])
station_list

['서울특별시 중구 세종대로 지하 2 (남대문로 5가)',
 '서울특별시 중구 세종대로 지하 101 (정동)',
 '서울특별시 종로구 종로 지하 55 (종로1가)',
 '서울특별시 종로구 종로 지하 129 (종로3가)',
 '서울특별시 종로구 종로 지하 216  (종로5가)',
 '서울특별시 종로구 종로 지하 302 (창신동)',
 '서울특별시 동대문구 왕산로 지하 1 (신설동)',
 '서울특별시 동대문구 왕산로 지하 93 (제기동)',
 '서울특별시 동대문구 왕산로 지하 205(전농동)',
 '서울특별시 종로구 종로 359 (숭인동)',
 '서울특별시 강서구 강서로 지하54 (화곡동)',
 '서울특별시 중구 을지로 지하 42 (을지로1가)',
 '서울특별시 중구 을지로 지하 106 (을지로3가)',
 '서울특별시 중구 을지로 지하 178 (을지로4가)',
 '서울특별시 중구 을지로 지하 279 (을지로7가)',
 '서울특별시 중구 퇴계로 지하 431-1 (신당동)',
 '서울특별시 성동구 왕십리로 지하 374 (하왕십리동)',
 '서울특별시 성동구 왕십리로 지하300 (행당동)',
 '서울특별시 성동구 왕십리로 206 (행당동)',
 '서울특별시 성동구 아차산로 18 (성수동1가)',
 '서울특별시 성동구 아차산로 100 (성수동2가)',
 '서울특별시 광진구 아차산로 243 (화양동)',
 '서울특별시 광진구 아차산로 384-1 (구의동)',
 '서울특별시 광진구 강변역로 53 (구의동)',
 '서울특별시 송파구 오금로 20 (신천동)',
 '서울특별시 송파구 올림픽로 지하 265 (잠실동)',
 '서울특별시 송파구 올림픽로 지하 140 (잠실동)',
 '서울특별시 송파구 올림픽로 지하 23 (잠실동)',
 '서울특별시 강남구 테헤란로 지하 538 (삼성동)',
 '서울특별시 강남구 테헤란로 지하 340 (삼성동)',
 '서울특별시 강남구 테헤란로 지하 156 (역삼동)',
 '서울특별시 강남구 강남대로 지하 396 (역삼동)',
 '서울특별시 서초구 서초

## Regex 적용으로 주소 데이터 정제

참고 사이트

[Regexr.com](https://regexr.com/)

In [21]:
import re

# 코드입력
# Regular Expression 적용하여 ()괄호를 제거 후 추출
pattern_string = r'.+(?=\()'
pattern = re.compile(pattern_string)

In [22]:
pattern.match('서울특별시 중구 세종대로 지하 2 (남대문로 5가)').group().strip()

'서울특별시 중구 세종대로 지하 2'

## 주소 데이터 클린징

In [23]:
def clean_address(x):
    # 코드입력
    location = pattern.match(x)
    if location:
        loc = location.group().strip()
    else:
        loc = x
    return loc   

`clean_address` 적용

In [24]:
# 코드입력
address['주소'] = address['역주소'].apply(clean_address)
address.head()

연번  역번호  호선    역명         역전화번호                           역주소  \
0   1  150   1   서울역  02-6110-1331  서울특별시 중구 세종대로 지하 2 (남대문로 5가)   
1   2  151   1    시청  02-6110-1321     서울특별시 중구 세종대로 지하 101 (정동)   
2   3  152   1    종각  02-6110-1311     서울특별시 종로구 종로 지하 55 (종로1가)   
3   4  153   1  종로3가  02-6110-1301    서울특별시 종로구 종로 지하 129 (종로3가)   
4   5  154   1  종로5가  02-6110-1291   서울특별시 종로구 종로 지하 216  (종로5가)   

                     주소  
0    서울특별시 중구 세종대로 지하 2  
1  서울특별시 중구 세종대로 지하 101  
2    서울특별시 종로구 종로 지하 55  
3   서울특별시 종로구 종로 지하 129  
4   서울특별시 종로구 종로 지하 216

### geocode 변환 및 위/경도 데이터 생성

geocode 변환 함수화

In [25]:
def geocode(x):
    # 코드입력
    results = maps.geocode(x)
    if len(results) > 0:
        result = results[0]
        return result['geometry']['location']['lat'], result['geometry']['location']['lng']
    else:
        return np.nan        

샘플 데이터 변환

In [26]:
print(station_list[1])
geocode(station_list[1])

서울특별시 중구 세종대로 지하 101 (정동)


(37.5665684, 126.9728716)

위경도 좌표계로 변환합니다. (tuple 형식으로 반환)

In [27]:
# 코드입력
address['위경도'] = address['주소'].apply(geocode)

위경도로 변환되지 않는 값 확인(결측치 확인)

In [28]:
address.loc[address['위경도'].isnull()]

Empty DataFrame
Columns: [연번, 역번호, 호선, 역명, 역전화번호, 역주소, 주소, 위경도]
Index: []

In [29]:
address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 0 to 291
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      253 non-null    int64 
 1   역번호     253 non-null    int64 
 2   호선      253 non-null    int64 
 3   역명      253 non-null    object
 4   역전화번호   253 non-null    object
 5   역주소     253 non-null    object
 6   주소      253 non-null    object
 7   위경도     253 non-null    object
dtypes: int64(3), object(5)
memory usage: 25.9+ KB


`lat`와 `lng` 컬럼 생성 후 위경도 데이터 분리하여 대입

In [30]:
# 코드입력
address['lat'] = address['위경도'].str[0]
address['lng'] = address['위경도'].str[1]

In [31]:
address.head()

연번  역번호  호선    역명         역전화번호                           역주소  \
0   1  150   1   서울역  02-6110-1331  서울특별시 중구 세종대로 지하 2 (남대문로 5가)   
1   2  151   1    시청  02-6110-1321     서울특별시 중구 세종대로 지하 101 (정동)   
2   3  152   1    종각  02-6110-1311     서울특별시 종로구 종로 지하 55 (종로1가)   
3   4  153   1  종로3가  02-6110-1301    서울특별시 종로구 종로 지하 129 (종로3가)   
4   5  154   1  종로5가  02-6110-1291   서울특별시 종로구 종로 지하 216  (종로5가)   

                     주소                        위경도        lat         lng  
0    서울특별시 중구 세종대로 지하 2  (37.5641186, 126.9768084)  37.564119  126.976808  
1  서울특별시 중구 세종대로 지하 101  (37.5657037, 126.9768616)  37.565704  126.976862  
2    서울특별시 종로구 종로 지하 55    (37.570169, 126.983099)  37.570169  126.983099  
3   서울특별시 종로구 종로 지하 129  (37.5703709, 126.9910053)  37.570371  126.991005  
4   서울특별시 종로구 종로 지하 216  (37.5597851, 126.9709861)  37.559785  126.970986

In [32]:
address[['호선', '역명', 'lat', 'lng', '주소', '역번호']]

호선      역명        lat         lng                    주소   역번호
0     1     서울역  37.564119  126.976808    서울특별시 중구 세종대로 지하 2   150
1     1      시청  37.565704  126.976862  서울특별시 중구 세종대로 지하 101   151
2     1      종각  37.570169  126.983099    서울특별시 종로구 종로 지하 55   152
3     1    종로3가  37.570371  126.991005   서울특별시 종로구 종로 지하 129   153
4     1    종로5가  37.559785  126.970986   서울특별시 종로구 종로 지하 216   154
..   ..     ...        ...         ...                   ...   ...
285   9    석촌고분  37.501983  127.096596     서울특별시 송파구 삼학사로 53  4132
287   9    송파나루  37.510080  127.113112     서울특별시 송파구 오금로 165  4134
288   9    한성백제  37.516359  127.113315    서울특별시 송파구 위례성대로 51  4135
290   9    둔촌오륜  37.524990  127.120881    서울특별시 송파구 강동대로 327  4137
291   9  중앙보훈병원  37.529317  127.142437     서울특별시 강동구 명일로 117  4138

[253 rows x 6 columns]

## 파일로 변환 (.csv로 내보내기)

In [33]:
address[['호선', '역명', 'lat', 'lng', '주소', '역번호']].to_csv('seoul_station_geocode.csv', index=False)

In [34]:
pd.read_csv('https://raw.githubusercontent.com/teddylee777/teddylee777.github.io/gh-pages/assets/files/seoul_station_geocode.csv')

호선      역명        lat         lng                    주소   역번호
0     1     서울역  37.564119  126.976808    서울특별시 중구 세종대로 지하 2   150
1     1      시청  37.565704  126.976862  서울특별시 중구 세종대로 지하 101   151
2     1      종각  37.570169  126.983099    서울특별시 종로구 종로 지하 55   152
3     1    종로3가  37.570436  126.992148   서울특별시 종로구 종로 지하 129   153
4     1    종로5가  37.572590  126.999912   서울특별시 종로구 종로 지하 216   154
..   ..     ...        ...         ...                   ...   ...
248   9    석촌고분  37.501983  127.096596     서울특별시 송파구 삼학사로 53  4132
249   9    송파나루  37.510080  127.113112     서울특별시 송파구 오금로 165  4134
250   9    한성백제  37.516317  127.113520    서울특별시 송파구 위례성대로 51  4135
251   9    둔촌오륜  37.524990  127.120881    서울특별시 송파구 강동대로 327  4137
252   9  중앙보훈병원  37.529317  127.142437     서울특별시 강동구 명일로 117  4138

[253 rows x 6 columns]